In [1]:
!pip install transformers
!pip install faiss-cpu
!pip install sentence-transformers


In [2]:
from transformers import pipeline
import retriever  # Importing the retriever functions from retriever.py


In [3]:
# Initialize the question-answering model
model_path = "radlab/polish-qa-v2"
question_answerer = pipeline("question-answering", model=model_path)

# Paths to the FAISS index and metadata file
faiss_path = '/content/130_len_faiss_index.bin'
metadata_path = '/content/130_len_metadata.json'

# Initialize retriever components
index, metadata, retriever_model = retriever.initialize_retriever(faiss_path, metadata_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.59M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

FAISS index loaded successfully.
Metadata loaded successfully.
Retriever model initialized successfully.


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
def generate_answer(question):
    """Generates an answer by retrieving relevant chunks and passing them to the question-answering model."""
    # Retrieve relevant chunks for the question
    top_chunks = retriever.retrieve_top_chunks(
        query=question,
        index=index,
        metadata=metadata,
        retriever_model=retriever_model,
        initial_top_k=50,
        final_top_k=5
    )

    # Combine retrieved text chunks into a single context
    context = ' '.join([chunk['text'] for chunk in top_chunks])

    # Generate answer using the question-answering model
    answer = question_answerer(
        question=question,
        context=context
    )

    return answer['answer']

# TES

In [5]:
# Test questions
questions = [
    "Co jest najważniejsze dla pracowników?",
    "Dlaczego ważna jest odpowiedzialność w biznesie?"
]

# Generate answers for each question
for question in questions:
    print(f"\nQuestion: {question}")
    answer = generate_answer(question)
    print(f"Answer: {answer}")


Question: Co jest najważniejsze dla pracowników?
Query: co jest najważniejsze dla pracowników
Retrieved in 0.1988 seconds
Average similarity distance of results: 1.0523
Results count after filtering: 5

Answer:  zaangażowanie

Question: Dlaczego ważna jest odpowiedzialność w biznesie?
Query: dlaczego ważna jest odpowiedzialność w biznesie
Retrieved in 0.1398 seconds
Average similarity distance of results: 1.1836
Results count after filtering: 5

Answer:  więcej mam odpowiedzialności, tym jest ten ważniejszy.
